In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error

c:\Users\Leo90\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\Leo90\anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
c:\Users\Leo90\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
import os
import ssl
from cryptography import x509
from cryptography.hazmat.backends import default_backend
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import random
# !unzip /content/drive/MyDrive/research/certs.zip

'unzip' ���O�����Υ~���R�O�B�i���檺�{���Χ妸�ɡC


In [3]:
certs = ["benign-cert", "malicious-cert"]
# cert_texts including certificate's subject and issuer
cert_texts = []
# cert_labels represent benign or malicious
cert_labels = []
for cert in certs:
    count = 0
    for filename in os.listdir(cert):
        file_path = os.path.join(cert, filename)
        # read file in binary mode cause load_pem_x509_certificate takes bytes
        with open(file_path, 'rb') as cert_file:
            texts = ''
            pem_cert = cert_file.read()
            # load PEM format certificate into x509 object
            try:
                x509_cert = x509.load_pem_x509_certificate(pem_cert, default_backend())
                count+=1
                if count == 3000:
                  break
            except:
                continue
            # extract subject and issuer principal from x509 object
            try:
                texts += x509_cert.subject.get_attributes_for_oid(x509.NameOID.COMMON_NAME)[0].value
            except:
                texts += 'N/A'
            texts += ' '
            issuer_attr = [x509.NameOID.COUNTRY_NAME, x509.NameOID.ORGANIZATION_NAME, x509.NameOID.COMMON_NAME]
            for attr in issuer_attr:
                try:
                    texts += x509_cert.issuer.get_attributes_for_oid(attr)[0].value
                except:
                    texts += 'N/A'
                texts += ' '
            # print(texts)
            cert_texts.append(texts)
            cert_labels.append(1 if cert == certs[0] else 0)
            # extension = x509_cert.extensions.get_extension_for_oid(x509.ExtensionOID.EXTENDED_KEY_USAGE).value
            # print(extension)
            # extended_key_usage_names = [oid._name for oid in extension]
            # print(extended_key_usage_names)

# cert_texts & cert_labels length = 12086
# zipped = list(zip(cert_texts, cert_labels))
# random.shuffle(zipped)
# a, b = zip(*zipped)
# cert_texts = list(a)
# cert_labels = list(b)

FileNotFoundError: [WinError 3] 系統找不到指定的路徑。: '/content/certs/benign-cert'

In [ ]:
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
            self.texts = texts
            self.labels = labels
            self.tokenizer = tokenizer
            self.max_length = max_length
    def __len__(self):
        return len(self.texts)
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        # input_ids encodes input texts into tokenized integer sequence
        # attention_mask is used to ignore padded tokens, only attend to real tokens
        # torch.tensor(label) convert label into pytorch tensor
        encoding = self.tokenizer(text, return_tensors='pt', max_length=self.max_length, padding='max_length', truncation=True)
        return {'input_ids': encoding['input_ids'].flatten(), 'attention_mask': encoding['attention_mask'].flatten(), 'label': torch.tensor(label)}

class BERTClassifier(nn.Module):
    def __init__(self, bert_model_name, num_classes):
        super(BERTClassifier, self).__init__()
        # load pre-trained bert model
        self.bert = BertModel.from_pretrained(bert_model_name)
        # initialize dropout layer to prevent overfitting, dropout rate = 0.1
        self.dropout = nn.Dropout(0.1)
        #* initialize fully connected layer, input size = bert.config.hidden_size, output size = num_classes
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)
    def forward(self, input_ids, attention_mask):
        # feed input_ids and attention_mask into bert model
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        # extract the last hidden state of the first token, which is [CLS] token
        pooled_output = outputs.pooler_output
        # feed pooled_output into dropout layer to prevent overfitting
        x = self.dropout(pooled_output)
        # feed pooled_output into fully connected layer to produce the final logits,
        # which represent the unnormalized scores for each class
        logits = self.fc(x)
        return logits

def train(model, data_loader, optimizer, scheduler, device):
    # set model to training mode
    model.train()
    for batch in data_loader:
        # PyTorch accumulates gradients, so we need to clear them out before each batch
        optimizer.zero_grad()
        # move batch to device
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        # feed input_ids and attention_mask into model to get logits
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        # calculate loss between logits and labels
        # Cross-entropy loss is commonly used for classification tasks
        # nn.CrossEntropyLoss() combines nn.LogSoftmax() and nn.NLLLoss() in a single class
        loss = nn.CrossEntropyLoss()(outputs, labels)
        # calculate gradients of model parameters with respect to loss
        loss.backward()
        # optimizer.step() performs a parameter update based on the current gradient
        optimizer.step()
        # scheduler.step() update learning rate
        scheduler.step()

def evaluate(model, data_loader, device):
    # set model to evaluation mode
    model.eval()
    predictions = []
    actual_labels = []
    # disable gradient calculation to save memory and computation
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            # print(outputs)
            # torch.max() returns the maximum value of each row of the input tensor in the given dimension dim
            _, preds = torch.max(outputs, dim=1)
            # append predictions and actual labels to calculate accuracy and classification report
            predictions.extend(preds.cpu().tolist())
            actual_labels.extend(labels.cpu().tolist())
    return accuracy_score(actual_labels, predictions),\
            classification_report(actual_labels, predictions),\
            mean_squared_error(actual_labels, predictions)

In [ ]:
# Set up parameters
bert_model_name = 'bert-base-uncased'
num_classes = 2
max_length = 128
batch_size = 16
num_epochs = 8
learning_rate = 2e-5

train_texts, eval_texts, train_labels, eval_labels = \
train_test_split(cert_texts, cert_labels, test_size=0.4, random_state=42)
# print(len(train_texts))
tokenizer = BertTokenizer.from_pretrained(bert_model_name)
train_dataset = TextClassificationDataset(train_texts, train_labels, tokenizer, max_length)
eval_dataset = TextClassificationDataset(eval_texts, eval_labels, tokenizer, max_length)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
eval_dataloader = DataLoader(eval_dataset, batch_size=batch_size)

# torch.cuda.current_device()
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(device)
model = BERTClassifier(bert_model_name, num_classes).to(device)
# print(torch.version.cuda)

# AdamW is a class from the huggingface library (as opposed to pytorch)
optimizer = AdamW(model.parameters(), lr=learning_rate)
# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * num_epochs
# Create the learning rate scheduler. This changes the learning rate as the training loop progresses
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    train(model, train_dataloader, optimizer, scheduler, device)
    accuracy, report, mse = evaluate(model, eval_dataloader, device)
    print(f"Validation Accuracy: {accuracy}")
    print(f"mean_squared_error: {mse}")
    print(report)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/8
Validation Accuracy: 0.9744346116027532
mean_squared_error: 0.025565388397246803
              precision    recall  f1-score   support

           0       0.98      0.96      0.97       811
           1       0.97      0.99      0.98      1223

    accuracy                           0.97      2034
   macro avg       0.97      0.97      0.97      2034
weighted avg       0.97      0.97      0.97      2034

Epoch 2/8
Validation Accuracy: 0.9709931170108161
mean_squared_error: 0.029006882989183875
              precision    recall  f1-score   support

           0       0.98      0.94      0.96       811
           1       0.96      0.99      0.98      1223

    accuracy                           0.97      2034
   macro avg       0.97      0.97      0.97      2034
weighted avg       0.97      0.97      0.97      2034

Epoch 3/8
Validation Accuracy: 0.967551622418879
mean_squared_error: 0.032448377581120944
              precision    recall  f1-score   support

           0       